# Stack Overflow Python Questions/Answers Building 

## Import and check total dataset

In [1]:
from google.cloud import bigquery

In [2]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="config.json"

In [3]:
client = bigquery.Client()

In [4]:
dataset_ref = client.dataset('stackoverflow', project='bigquery-public-data')

In [5]:
type(dataset_ref)

google.cloud.bigquery.dataset.DatasetReference

In [6]:
dset = client.get_dataset(dataset_ref)

In [7]:
type(dset)

google.cloud.bigquery.dataset.Dataset

In [8]:
[x.table_id for x in client.list_tables(dset)]

['badges',
 'comments',
 'post_history',
 'post_links',
 'posts_answers',
 'posts_moderator_nomination',
 'posts_orphaned_tag_wiki',
 'posts_privilege_wiki',
 'posts_questions',
 'posts_tag_wiki',
 'posts_tag_wiki_excerpt',
 'posts_wiki_placeholder',
 'stackoverflow_posts',
 'tags',
 'users',
 'votes']

In [9]:
full_questions = client.get_table(dset.table('posts_questions'))

In [10]:
full_answers = client.get_table(dset.table('posts_answers'))

## Building questions dataset

In [11]:
full_questions.schema

[SchemaField('id', 'INTEGER', 'NULLABLE', None, ()),
 SchemaField('title', 'STRING', 'NULLABLE', None, ()),
 SchemaField('body', 'STRING', 'NULLABLE', None, ()),
 SchemaField('accepted_answer_id', 'INTEGER', 'NULLABLE', None, ()),
 SchemaField('answer_count', 'INTEGER', 'NULLABLE', None, ()),
 SchemaField('comment_count', 'INTEGER', 'NULLABLE', None, ()),
 SchemaField('community_owned_date', 'TIMESTAMP', 'NULLABLE', None, ()),
 SchemaField('creation_date', 'TIMESTAMP', 'NULLABLE', None, ()),
 SchemaField('favorite_count', 'INTEGER', 'NULLABLE', None, ()),
 SchemaField('last_activity_date', 'TIMESTAMP', 'NULLABLE', None, ()),
 SchemaField('last_edit_date', 'TIMESTAMP', 'NULLABLE', None, ()),
 SchemaField('last_editor_display_name', 'STRING', 'NULLABLE', None, ()),
 SchemaField('last_editor_user_id', 'INTEGER', 'NULLABLE', None, ()),
 SchemaField('owner_display_name', 'STRING', 'NULLABLE', None, ()),
 SchemaField('owner_user_id', 'INTEGER', 'NULLABLE', None, ()),
 SchemaField('parent_id'

In [12]:
QUERY_QUESTIONS = """
SELECT 
  q.id, 
  q.title, 
  q.body, 
  q.tags,
  q.score,
  q.accepted_answer_id,
  q.answer_count
FROM `bigquery-public-data.stackoverflow.posts_questions` AS q  
WHERE q.tags LIKE '%python%'
ORDER BY q.answer_count DESC
LIMIT 300000
"""

data_query = client.query(QUERY_QUESTIONS)
rows = data_query.result()

In [13]:
! rm questions_data.csv answers_data.csv

rm: cannot remove 'questions_data.csv': No such file or directory
rm: cannot remove 'answers_data.csv': No such file or directory


In [14]:
import csv

In [15]:
q_ids = []
with open('questions_data.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['id', 'title', 'body', 'tags', 'score', 'accepted_answer_id', 'answer_count'])
    for row in rows:
      qid = row.id
      q_ids.append(str(qid))
      writer.writerow([qid, row.title, row.body, row.tags, row.score, row.accepted_answer_id, row.answer_count])

In [16]:
print("number of question: " + str(len(q_ids)))

number of question: 300000


## Building corresponding answers dataset

In [17]:
full_answers.schema

[SchemaField('id', 'INTEGER', 'NULLABLE', None, ()),
 SchemaField('title', 'STRING', 'NULLABLE', None, ()),
 SchemaField('body', 'STRING', 'NULLABLE', None, ()),
 SchemaField('accepted_answer_id', 'STRING', 'NULLABLE', None, ()),
 SchemaField('answer_count', 'STRING', 'NULLABLE', None, ()),
 SchemaField('comment_count', 'INTEGER', 'NULLABLE', None, ()),
 SchemaField('community_owned_date', 'TIMESTAMP', 'NULLABLE', None, ()),
 SchemaField('creation_date', 'TIMESTAMP', 'NULLABLE', None, ()),
 SchemaField('favorite_count', 'STRING', 'NULLABLE', None, ()),
 SchemaField('last_activity_date', 'TIMESTAMP', 'NULLABLE', None, ()),
 SchemaField('last_edit_date', 'TIMESTAMP', 'NULLABLE', None, ()),
 SchemaField('last_editor_display_name', 'STRING', 'NULLABLE', None, ()),
 SchemaField('last_editor_user_id', 'INTEGER', 'NULLABLE', None, ()),
 SchemaField('owner_display_name', 'STRING', 'NULLABLE', None, ()),
 SchemaField('owner_user_id', 'INTEGER', 'NULLABLE', None, ()),
 SchemaField('parent_id', '

In [18]:
# query more than 1024k characters is not supported, so broken it into 20 parts

def get_qids_str(start):
  q_ids_list = ", ".join((q_ids[start: start+10000]))
  return "(" + q_ids_list + ")"

In [19]:
def get_rows(qids_list):
  QUERY_ANSWERS = """
  SELECT 
    a.id,
    a.parent_id,
    a.body, 
    a.score
  FROM `bigquery-public-data.stackoverflow.posts_answers` AS a 
  WHERE a.parent_id IN {}
  """.format(qids_list)

  data_query = client.query(QUERY_ANSWERS)
  return data_query.result()

In [20]:
with open('answers_data.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['id', 'parent_id', 'body', 'tags', 'score'])
    for i in range(0, 200000, 10000):
      rows = get_rows(get_qids_str(i))
      # print(rows.total_rows)
      for row in rows:
        writer.writerow([row.id, row.parent_id, row.body, row.score])

## Checking the built datasets

In [21]:
import pandas as pd

In [22]:
dataset_questions = pd.read_csv('questions_data.csv')

In [23]:
dataset_questions

,id,title,body,tags,score,accepted_answer_id,answer_count
0,101268,Hidden features of Python,<p>What are the lesser-known but useful featur...,python|hidden-features,1418,NaN,191
1,3088,Best ways to teach a beginner to program?,<p><strong>Original Question</strong></p>\n\n<...,python|language-agnostic,324,NaN,86
2,312443,How do you split a list into evenly sized chunks?,"<p>I have a list of arbitrary length, and I ne...",python|list|split|chunks,2409,312464.0,64
3,89228,How to call an external command?,<p>How do you call an external command (as if ...,python|shell|terminal|subprocess|command,5096,89243.0,62
4,550632,Favorite Django Tips & Features?,<p>Inspired by the question series 'Hidden fea...,python|django|hidden-features,308,NaN,55
...,...,...,...,...,...,...,...
299995,64932738,CNN in Pytorch: labels for a custom dataset,<p>I have a very basic question that would be ...,python|deep-learning|pytorch|cnn,0,64940473.0,2
299996,4413763,"Python in java, is it possible",<p>I have a class that is written in Java.<br>...,java|python,0,4413778.0,2
299997,60490366,TypeError: string indices must be integers wit...,<p>I want to compare 2 dataframe rows to eacho...,python|pandas|dataframe,0,60490449.0,2
299998,4413798,python restart the program after running a method,"<p>A noob question I'm sure.\nFor example, say...",python,0,4413827.0,2


In [24]:
dataset_answers = pd.read_csv('answers_data.csv')

In [25]:
dataset_answers

,id,parent_id,body,tags,score
0,14803823,279237,<p>The easiest way without any modification to...,23,NaN
1,15137914,4071396,<p>I came to add:</p>\n\n<p><code>map(str.stri...,24,NaN
2,14733488,9577012,<p>What Celery is doing is very much akin to <...,27,NaN
3,14917111,14917092,<p>You need to use <code>isinstance</code> to ...,28,NaN
4,14700501,1720421,<p>It's worth noting that the <code>itertools....,29,NaN
...,...,...,...,...,...
810574,46475985,46475777,"<p>Use <a href=""http://www.regular-expressions...",4,NaN
810575,46621777,46621712,<p>You can use <code>df.loc[_not_yet_existing_...,50,NaN
810576,46415358,46415260,<p>Sorry if this seems obvious but there's not...,0,NaN
810577,46704992,46703873,<p>The type of error you are observing seems t...,1,NaN
